# Text Analytics Group Assignment2
Due: September 25, 2017, by 11:59 p.m. In this assignment you are an analytics consultant to a   (i) brand manager, (ii) product manager and (iii)advertising manager. Your job is    to give advice/insights to these individuals based on the analysis of social media conversations. The detailed tasks are described below. We use cars as an example of a “high involvement” good (recall from class discussions that for high involvement goods, people use social media heavily for awareness building and research).

1.Develop a   crawler/scraper to fetch messages posted in Edmunds.com discussion forums. The crawler output should be a .csv file with the following columns: date, userid, and message (even though you will only use the messages in your analysis). Before you develop the crawler, carefully study one of the forums on Edmunds.com to understand the html as well as the threading structures.

2.Fetch between 5,000 and 10,000 posts about cars from a General topics forum. Do NOT choosea forum dedicated to a   particular brand or model. Instead, you can choose the General & Sedans categories and then select, for example, the Entry Level Luxury forum https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedansThe idea is    to have multiple brands and models being discussed without one of them being the focal point.

3.Once you fetch the data, find the top 10 brands from frequency counts. You will need to write a script to count the frequencies. Be sure not to count a mention more than once per post, evenif   it    is mentioned multiple times.Replace modelswith brands so that from now on you have to deal with only brands and not models. You will need another script for this job. This step is meant to help simplify the analysis. A    list of model and brand names (not exhaustive) are provided in a separate file.

#### Task A:
Identify top 10 brands by frequency. From the posts, calculate lift ratios for associations between the brands. You will have to write a   script to do this task). Show the brands on a   multi-dimensional scaling (MDS) map (use a Python script for MDS, there are multiple scripts available onthe Internet). 

In [4]:
import pandas as pd

In [41]:
#car forum data
forum_data =pd.read_csv('reed_new_out.csv', header = 0)
forum_data = forum_data[['date','userid','message']]

df2=pd.DataFrame([['April 12, 2002 2:53PM','dani',
                   'audi bmw acura nissan volkswagen honda toyota chrysler mercedes subaru'
                   'bmw acura audi infiniti honda nissan toyota ford mercedes subaru'
                  ]], columns=['date','userid','message'])
forum_data=forum_data.append(df2)
forum_data['message']=forum_data['message'].str.lower()
forum_data[:3]
len(forum_data['message'].value_counts())

5750

In [43]:
#list of car brands/names
model_list_utf =pd.read_csv('reed_Cars make model list.csv', encoding='utf-8', names=['Replace', 'Search'])
model_list_utf.Replace=model_list_utf.Replace.str.lower()
model_list_utf.Search=model_list_utf.Search.str.lower()
model_list_utf[:3]

,Replace,Search
0,model replacement,NaN
1,NaN,NaN
2,replace,search


In [45]:
#formula to replace model names with brand names
#split each message into words. if word in search values, replace with matching idx number?
import numpy as np
def replace_model(s):
    mess=s.split()
    mess_rep=[]
    for word in mess:
        if word in model_list_utf['Search'].values:
            mask=(model_list_utf['Search']==word)
            word=model_list_utf[mask]['Replace'].values[0]
            mess_rep.append(word)
        else:
            mess_rep.append(word)
    new_mess=' '.join(mess_rep)        
    return new_mess    


In [54]:
"u'integra'" in 'integral hey'

False

In [51]:
model_list_utf['Search'].values[3]

u'integra'

In [49]:
import re
def replace_model(s):
    for model in model_list_utf['Search'].values:
        if model in s:
            
        
        matches = re.findall(model,s)
        if len(matches)>0:
            re.sub(model, model_list_utf[model_list_utf['Search']==model]['Replace'].values[0],s)
        else:
            s=s
    return s
replace_model("98a4quattro 2.8, new 325ci,325xi'")

TypeError: first argument must be string or compiled pattern

In [44]:
#mapping that function to forum df
forum_data['message']=forum_data['message'].astype(str)
forum_data['clean_message']=forum_data['message'].map(replace_model)
forum_data[:3]

,date,userid,message,clean_message
0,"March 23, 2002 7:04PM",cybersol,entry level performance luxury sedans are a ho...,entry level performance luxury sedan are a hot...
1,"March 25, 2002 5:54AM",merc1,i personally think that with a few tweaks the ...,i personally think that with a few tweaks the ...
2,"March 25, 2002 7:06AM",fredvh,i am debating a new purchase and these two are...,i am debating a new purchase and these two are...


In [47]:
mask=forum_data['clean_message'].str.contains('a4')
forum_data[mask]['clean_message'][64]

"i didn't want a manual tranny because i like to eat a sandwich or be on the phone, roll up/down windows and sunroof at any time while i drive. i test-drove a used '98a4quattro 2.8, new 325ci,325xi. i loved those bimmers and audi was a great car but the bottom line sticker is what made me go with the acura. with the steptronic this car is an absolute joy, i went from a 1993dodge p.o.s. i'm only at 150miles so far but no complaints and extremely satisified with my decision..."

In [72]:
#removing stopwords and adding to new column in df
from nltk.corpus import stopwords
stop = stopwords.words('english')
forum_data['message'] = forum_data['message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [73]:
forum_data

,date,userid,message
0,"March 23, 2002 7:04PM",cybersol,entry level performance luxury sedan hot area ...
1,"March 25, 2002 5:54AM",merc1,personally think tweaks c320 could also sit to...
2,"March 25, 2002 7:06AM",fredvh,debating new purchase two top list. price diff...
3,"March 25, 2002 5:02PM",blueguydotcom,"great handling, rwd, excellent engine best war..."
4,"March 25, 2002 11:04PM",hungrywhale,"manual tranny. may matter you, certainly matte..."
5,"March 26, 2002 12:44AM",riez,one beauty bmw 3 series many models choose fro...
6,"March 26, 2002 8:20PM",blueguydotcom,"good grief, wait 9 months manual. even auto, p..."
7,"March 26, 2002 9:02PM",hungrywhale,"i'll give fair shot manual comes out, i'm flat..."
8,"March 26, 2002 9:45PM",blueguydotcom,understand it's speed. really don't care car w...
9,"March 27, 2002 12:00AM",hungrywhale,sounds like know won't even put list (let alon...


In [49]:
#count the number of messages that contain each brand name
#careful- don't want to count same message twice if brand is stated twice

counter={}
for brand in model_list['Replace'].drop_duplicates():
    if len(forum_data['message_without_stopwords'].str.contains(brand).value_counts().values)>1:
        count=forum_data['message_without_stopwords'].str.contains(brand).value_counts()[True]
    else:
        count=0
    counter[brand]=count
counter


mask=forum_data['message_without_stopwords'].str.contains('a4')
forum_data['message_without_stopwords'][mask].index #gives indices of rows that contain chrysler

Int64Index([  64,   69,   70,   74,   75,   77,   84,   90,  107,  124,
            ...
            5001, 5002, 5005, 5007, 5200, 5209, 5313, 5319, 5370, 5660],
           dtype='int64', length=107)

In [50]:
counterlist=[]
for key, value in counter.iteritems():
    temp = [key,value]
    counterlist.append(temp)
counterlist

sorted(counterlist, key=lambda x: x[1],reverse=True)
top10=sorted(counterlist, key=lambda x: x[1],reverse=True)[:10]
top10

[['bmw', 1768],
 ['acura', 611],
 ['audi', 523],
 ['infiniti', 485],
 ['honda', 405],
 ['nissan', 290],
 ['toyota', 266],
 ['ford', 253],
 ['mercedes', 236],
 ['subaru', 213]]

In [52]:
#dictionary of top 10 to values and list of top 10 brands
num={}
for i in top10:
    num[i[0]]=i[1]
num

listof10=[]
for i in top10:
    listof10.append(i[0])
listof10


['bmw',
 'acura',
 'audi',
 'infiniti',
 'honda',
 'nissan',
 'toyota',
 'ford',
 'mercedes',
 'subaru']

In [66]:
import itertools
list(itertools.combinations(listof10,2))
comb=list(itertools.combinations(listof10,2))
comb

[('bmw', 'acura'),
 ('bmw', 'audi'),
 ('bmw', 'infiniti'),
 ('bmw', 'honda'),
 ('bmw', 'nissan'),
 ('bmw', 'toyota'),
 ('bmw', 'ford'),
 ('bmw', 'mercedes'),
 ('bmw', 'subaru'),
 ('acura', 'audi'),
 ('acura', 'infiniti'),
 ('acura', 'honda'),
 ('acura', 'nissan'),
 ('acura', 'toyota'),
 ('acura', 'ford'),
 ('acura', 'mercedes'),
 ('acura', 'subaru'),
 ('audi', 'infiniti'),
 ('audi', 'honda'),
 ('audi', 'nissan'),
 ('audi', 'toyota'),
 ('audi', 'ford'),
 ('audi', 'mercedes'),
 ('audi', 'subaru'),
 ('infiniti', 'honda'),
 ('infiniti', 'nissan'),
 ('infiniti', 'toyota'),
 ('infiniti', 'ford'),
 ('infiniti', 'mercedes'),
 ('infiniti', 'subaru'),
 ('honda', 'nissan'),
 ('honda', 'toyota'),
 ('honda', 'ford'),
 ('honda', 'mercedes'),
 ('honda', 'subaru'),
 ('nissan', 'toyota'),
 ('nissan', 'ford'),
 ('nissan', 'mercedes'),
 ('nissan', 'subaru'),
 ('toyota', 'ford'),
 ('toyota', 'mercedes'),
 ('toyota', 'subaru'),
 ('ford', 'mercedes'),
 ('ford', 'subaru'),
 ('mercedes', 'subaru')]

In [54]:
mask=forum_data['message'].str.contains('chrysler')
forum_data['message'][mask] #gives indices of rows that contain chrysler


0       entry level performance luxury sedan hot area ...
1       personally think tweaks c320 could also sit to...
33      much debate subject. want reliable car buy jap...
36      cybersol... can't fathom would want go slushbo...
49      fast. merit "nationality based" debate. let ex...
128     riez-you missed target again. trunk space exam...
180     also check chrysler 300m, especially special v...
183     looked chrysler yet? nice. drove one 3 years (...
188     sorry comparison know aren't same... comparing...
190     chrysler 3 years - problems. i'd say chrysler ...
201     strongly believe segmentation based actually p...
332     saugatak... real. drive a...fwd lexus es330 rw...
520     emergency business trip today. 120 miles state...
561     driving wife&#146;s honda chrysler 4-cyl i95 d...
926     saying performance package another $4000 added...
1079    germans spewing far unreliable crap detroit da...
1080    amazing, chrysler spewing unreliable crap conn...
1081    "why e

In [64]:
#need how many messages each combination occurs in
sum(forum_data['message'].str.contains('acura')) 
#counts the number of trues/number of messages that contains acura
forum_data['message'].str.contains('acura' and 'honda')

counter_comb=[]
for brand in comb:
    mask_b0=(forum_data['message'].str.contains(brand[0]))
    mask_b1=(forum_data['message'].str.contains(brand[1]))
    mask=(mask_b0 & mask_b1)
    count=len(forum_data['message'][mask])
    counter_comb[brand]=count
counter_comb


TypeError: list indices must be integers, not tuple

In [56]:
#test df
messa=['audi bmw bmw bmw', 'audi chrysler', 'audi', 'bmw','chrysler']
df=pd.DataFrame(data=messa)
df

mask=(df[0].str.contains('chrysler'))
mask1=(df[0].str.contains('audi'))
mask2=(mask & mask1)
df[mask2]

counter_test={}
for brand in ['audi','bmw','chrysler']:
    if len(df[0].str.contains(brand).value_counts().values)>1:
        count=df[0].str.contains(brand).value_counts()[True]
    else:
        count=0
    counter_test[brand]=count
print counter_test #works

counter_comb_test={}
for brand in [('audi','bmw'),('audi','chrysler'),('bmw','chrysler')]:
    mask_b0=(df[0].str.contains(brand[0]))
    mask_b1=(df[0].str.contains(brand[1]))
    mask=(mask_b0 & mask_b1)
    count=len(df[0][mask])
    counter_comb_test[brand]=count
print counter_comb_test #works

def lift_test(a,b):
    num_a_and_b=float(counter_comb_test[(a,b)])
    num_a=float(counter_test[a])
    num_b=float(counter_test[b])
    lift=float(num_a_and_b/(num_a*num_b))
    return lift

lift_ratios_test={}
for brands in [('audi','bmw'),('audi','chrysler'),('bmw','chrysler')]:
    lift=lift_test(brands[0],brands[1])
    lift_ratios_test[brands]=lift
    print brands[0]
    print 'ht',brands[1]

print lift_ratios_test

{'bmw': 2, 'chrysler': 2, 'audi': 3}
{('bmw', 'chrysler'): 0, ('audi', 'chrysler'): 1, ('audi', 'bmw'): 1}
audi
ht bmw
audi
ht chrysler
bmw
ht chrysler
{('bmw', 'chrysler'): 0.0, ('audi', 'chrysler'): 0.16666666666666666, ('audi', 'bmw'): 0.16666666666666666}


In [61]:
#calculate lift for associations between the top 10 brands
#number of messages with a and b / (num messages with a) * (num messages with b)
# def lift(a,b):
a ='audi'
b='chrysler'
def lift(a,b):
    num_a_and_b=counter_comb[(a,b)]
    num_a=float(counter[a])
    num_b=float(counter[b])
    lift=float(num_a_and_b/(num_a*num_b))
    return lift

# lift_ratios={}
# for brands in comb:
#     lift=lift(brands[0],brands[1])
#     lift_ratios[brands]=lift

# lift('audi','chrysler')
counter_comb[('audi','chrysler')] #598
# counter['audi'] #1195
# counter['chrysler'] #598- that would mean every message that has chrysler has audi....

# lift('audi','bmw')
# counter_comb[('audi','bmw')] #996
# counter['audi'] #1195
# counter['bmw'] #996.. this is wrong

KeyError: ('audi', 'chrysler')

#### Task B:
What insights can you offer brand managers from your analysis in Task A    (choose two brands that you can offer the most interesting/useful insights for)? 

#### Task C:
What are 5   most frequently mentioned attributes of cars in the discussions? Note that the same attribute may be described by different words –   e.g., pick-up and acceleration may both referto a more general attribute, “performance”. You have to make suitable replacements. Now pick the 5 most frequently mentioned brands. Which attributes are most strongly associated with which of these 5   brands? You DON’T have to do a sentiment analysis for this assignment.
While BMW has claimed that they are the “ultimate driving machine”, is    that how people feel on Edmunds? Show your analysis.   

#### Task D:
What advice will you give to a (i) product manager, and (ii) marketing/advertising manager of these brands based on your analysis in Task C? For this assignment, you can assume the sentiment (e.g., that it   is    positive). 

#### Task E:
Which is the most aspirational brand in your data in terms of people actually wanting to buy or own? Describe your analysis. What are the business implications for this brand? Provide the following details in your write-up (I    am not setting a   strict page limit, but 3-4 pages, single-spaced, 11 font size should be enough with screenshots)

1.Which forum you chose (provide URL) 

2.Which 10 brands you chose –   provide the frequency table

3.Show all lift calculations in a   table.

4.Show the MDS map (put screenshots in your report)

5.State the 5   attributes you chose (again, a   table is    good here). 

6.For task E, provide all details of your analysis –   e.g., how you measured “aspirational” and howyou found the most aspirational brand.

7.Advice/insights based on your analysis for brand, product and advertising managers. Your submission should include all scripts as well as your answers to the questions above (generally speaking, I  won’t run these scripts, but if   the numbers don’t look right, I may run some of them). Also include the final data file after all replacements of models and attributes have been done.